### Bug Hunt: Invisible Shape Broadcasting

This bug  does not have an explicit error and is extremely difficult to find.

Instead, debug defensively -- insert
`import pdb; pdb.set_trace()` after the first line in `def forward(...)` and step line-by-line, checking the results  of every line.

In [1]:
!python ../src/download.py

n_item 3953
n_user 6041
n_featuers 9994
n_occu 21
n_rows 1000209


In [2]:
import numpy as np
fh = np.load('data/dataset.npz', allow_pickle=True)

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']
train_d = fh['train_dict']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']
test_d = fh['test_dict']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])


In [3]:
from abstract_model import AbstractModel

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl

from pytorch_lightning.loggers import TensorBoardLogger


def l2_regularize(array):
    return torch.sum(array ** 2.0)


class MF(AbstractModel):
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, c_bias=1.0, batch_size=128):
        super().__init__()
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        self.c_bias = c_bias
        self.batch_size = batch_size
        self.save_hyperparameters()
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        
        # We've added new terms here:
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
    
    def forward(self, inputs):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = inputs[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = inputs[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item)
        
        # Pull out biases
        bias_user = self.bias_user(user_id)
        bias_item = self.bias_item(item_id)
        biases = (self.bias + bias_user + bias_item)

        # Add bias prediction to the interaction prediction
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        return loss_mse, {"mse": loss_mse}
    
    def reg(self):
        # Add new regularization to the biases
        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        # Compute L2 reularization over user (P) and item (Q) matrices
        reg_user =  l2_regularize(self.user.weight) * self.c_vector
        reg_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        log = {"reg_user": reg_user, "reg_item": reg_item,
               "reg_bias_user": reg_bias_user, "reg_bias_item": reg_bias_item}
        total = reg_user + reg_item + reg_bias_user + reg_bias_item
        return total, log

In [8]:
from pytorch_lightning.logging import WandbLogger

k = 5
c_vector = 1e-5
c_bias = 5e-8
model = MF(n_user, n_item, k=k, c_bias=c_bias, c_vector=c_vector,
          batch_size=1024)
model.save_data(train_x, train_y, test_x, test_y)

# add a logger
logger = WandbLogger(name="02_mf", project="simple_mf")

trainer = pl.Trainer(max_epochs=100, logger=logger,
                     early_stop_callback=True,
                     progress_bar_refresh_rate=1) 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [9]:
trainer.fit(model)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable

  | Name      | Type      | Params
----------------------------------------
0 | user      | Embedding | 30 K  
1 | item      | Embedding | 19 K  
2 | bias_user | Embedding | 6 K   
3 | bias_item | Embedding | 3 K   


> <ipython-input-7-5fc30d0f27f3>(51)forward()
-> biases = (self.bias + bias_user + bias_item)


(Pdb)  l


 46  	
 47  	        # Pull out biases
 48  	        bias_user = self.bias_user(user_id)
 49  	        bias_item = self.bias_item(item_id)
 50  	        import pdb; pdb.set_trace()
 51  ->	        biases = (self.bias + bias_user + bias_item)
 52  	
 53  	        # Add bias prediction to the interaction prediction
 54  	        prediction = ui_interaction + biases
 55  	        return prediction
 56  	


(Pdb)  p ui_interaction


...

 46  	
 47  	        # Pull out biases
 48  	        bias_user = self.bias_user(user_id)
 49  	        bias_item = self.bias_item(item_id)
 50  	        import pdb; pdb.set_trace()
 51  ->	        biases = (self.bias + bias_user + bias_item)
 52  	
 53  	        # Add bias prediction to the interaction prediction
 54  	        prediction = ui_interaction + biases
 55  	        return prediction
 56  	


(Pdb)  


 57  	    def loss(self, prediction, target):
 58  	        # MSE error between target = R_ui and prediction = p_u * q_i
 59  	        loss_mse = F.mse_loss(prediction, target.squeeze())
 60  	        return loss_mse, {"mse": loss_mse}
 61  	
 62  	    def reg(self):
 63  	        # Add new regularization to the biases
 64  	        reg_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
 65  	        reg_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
 66  	        # Compute L2 reularization over user (P) and item (Q) matrices
 67  	        reg_user =  l2_regularize(self.user.weight) * self.c_vector


(Pdb)  q


BdbQuit: 